#Tokenizers
Copyright 2021,2022 Denis Rothman, MIT License

Reference 1 for word embedding:
https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

Reference 2 for cosine similarity:
SciKit Learn cosine similarity documentation

***Upload text.txt before running the Notebook***

#Pre-Requisistes

In [ ]:
#@title Pre-Requisistes
!pip install gensim==3.8.3
import nltk
nltk.download('punkt')

In [ ]:
import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 

#Word2Vec Tokenization

In [ ]:
#@title Word2Vec Tokenization
#‘text.txt’ file 
sample = open("text.txt", "r") 
s = sample.read() 

# processing escape characters 
f = s.replace("\n", " ") 

data = [] 
# sentence parsing
for i in sent_tokenize(f): 
	temp = [] 
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower())
	data.append(temp)

# Creating Skip Gram model 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 512,window = 5, sg = 1) 
print(model2)

Word2Vec(vocab=10816, size=512, alpha=0.025)


#Cosine Similarity

In [ ]:
#@title Cosine Similarity
def similarity(word1,word2):
        cosine=False #default value
        try:
                a=model2[word1]
                cosine=True
        except KeyError:     #The KeyError exception is raised
                print(word1, ":[unk] key not found in dictionary")#False implied

        try:
                b=model2[word2]#a=True implied
        except KeyError:       #The KeyError exception is raised
                cosine=False   #both a and b must be true
                print(word2, ":[unk] key not found in dictionary")

        if(cosine==True):
                b=model2[word2]
                # compute cosine similarity
                dot = np.dot(a, b)
                norma = np.linalg.norm(a)
                normb = np.linalg.norm(b)
                cos = dot / (norma * normb)

                aa = a.reshape(1,512) 
                ba = b.reshape(1,512)
                #print("Word1",aa)
                #print("Word2",ba)
                cos_lib = cosine_similarity(aa, ba)
                #print(cos_lib,"word similarity")
          
        if(cosine==False):cos_lib=0;
        return cos_lib

#Case 0: Words in the dataset and the dictionary

In [ ]:
#@title Case 0: Words in text and dictionary
word1="freedom";word2="liberty"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.8010981]] freedom liberty


#Case 1: Words not in the dataset or the dictionary

In [ ]:
#@title Word(s) Case 1: Word not in text or dictionary
word1="corporations";word2="rights"
print("Similarity",similarity(word1,word2),word1,word2)

corporations :[unk] key not found in dictionary
Similarity 0 corporations rights


#Case 2: Noisy Relationship

In [ ]:
#@title Case 2: Noisy Relationship 
word1="etext";word2="declaration"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.9052981]] etext declaration


#Case 3: Words in the text but not in the dictionary

In [ ]:
#@title Case 3: word in text, not in dictionary
word1="pie";word2="logic"
print("Similarity",similarity(word1,word2),word1,word2)

pie :[unk] key not found in dictionary
Similarity 0 pie logic


#Case 4: Rare words

In [ ]:
#@title Case 4: Rare words
word1="justiciar";word2="judgement"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.6612152]] justiciar judgement


#Case 5: Replacing rare words

In [ ]:
#@title Case 5: Replacing rare words
word1="judge";word2="judgement"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.7046789]] judge judgement


#Case 6: Entailment

In [ ]:
#@title Case 6: Entailment
word1="pay";word2="debt"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.9040462]] pay debt
